In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io
import glob
from PIL import Image
import utils

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights

weights = VGG16_Weights.DEFAULT
vgg_model = vgg16(weights=weights)

In [ ]:
for param in vgg_model.features[-8:].parameters():  # Unfreeze last 8 convolutional layers
    param.requires_grad = True

In [ ]:
N_CLASSES = 6
my_model = nn.Sequential(
    vgg_model.features,
    vgg_model.avgpool,
    nn.Flatten(),
    nn.Linear(512 * 7 * 7, 2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(2048, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(1024, N_CLASSES)
)

my_model = my_model.to(device)

In [ ]:
loss_function =nn.CrossEntropyLoss()
optimizer = Adam(my_model.parameters(), lr=0.0001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

In [ ]:
IMG_WIDTH, IMG_HEIGHT = (224, 224)
pre_trans = weights.transforms()

# Data Augmentation
random_trans = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
])

In [ ]:
DATA_LABELS = ["freshapples", "freshbanana", "freshoranges", "rottenapples", "rottenbanana", "rottenoranges"]

class MyDataset(Dataset):
    def __init__(self, data_dir, augment=False):
        self.imgs = []
        self.labels = []
        self.augment = augment

        for l_idx, label in enumerate(DATA_LABELS):
            data_paths = glob.glob(data_dir + label + '/*.png', recursive=True)
            for path in data_paths:
                img = tv_io.read_image(path, tv_io.ImageReadMode.RGB)
                img = pre_trans(img).to(device)
                self.imgs.append(img)
                self.labels.append(torch.tensor(l_idx).to(device))

    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]

        if self.augment:
            img = random_trans(img)
        return img, label

    def __len__(self):
        return len(self.imgs)

In [ ]:
n = 16
train_path = "data/fruits/train/"
train_data = MyDataset(train_path, augment=True)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

valid_path = "data/fruits/valid/"
valid_data = MyDataset(valid_path, augment=False)
valid_loader = DataLoader(valid_data, batch_size=n, shuffle=False)
valid_N = len(valid_loader.dataset)

In [ ]:
def validate(model, valid_loader, valid_N, loss_function):
    loss = 0
    correct = 0
    total = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            loss += loss_function(output, y).item()
            _, predicted = torch.max(output, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)

    accuracy = (correct / total) * 100
    print('Valid - Loss: {:.4f}, Accuracy: {:.2f}%'.format(loss / len(valid_loader), accuracy))
    return accuracy

In [ ]:
epochs = 50
desired_accuracy = 92

for epoch in range(epochs):
    print(f"Epoch: {epoch + 1}/{epochs}")
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    valid_accuracy = validate(my_model, valid_loader, valid_N, loss_function)

    # Check early stopping
    if valid_accuracy >= desired_accuracy:
        print(f"Stopping early as validation accuracy reached: {valid_accuracy:.2f}%")
        torch.save(my_model.state_dict(), "best_model.pth")
        break

    # Step scheduler
    scheduler.step()

print("Training complete!")

If models accuracy is not greater or equal to 92% then do the remaining task

In [ ]:
# Unfreeze the base model
vgg_model.requires_grad_(True)
optimizer = Adam(my_model.parameters(), lr=.0001)

In [ ]:
epochs = 5

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Is still not get that accuracy then again run the above cell only with the same epoch of 5 or contact me...